In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Fetch stock data
def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data


def compute_rsi(series, period=14):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def compute_macd(series, short_window=12, long_window=26, signal_window=9):
    short_ema = series.ewm(span=short_window, adjust=False).mean()
    long_ema = series.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

# Calculate technical indicators
def calculate_technical_indicators(stock_data):
    stock_data['SMA50'] = stock_data['Close'].rolling(window=50).mean()
    stock_data['SMA200'] = stock_data['Close'].rolling(window=200).mean()
    stock_data['RSI'] = compute_rsi(stock_data['Close'])
    stock_data['MACD'], stock_data['Signal'] = compute_macd(stock_data['Close'])
    return stock_data

# Generate trading signals
def generate_signals(stock_data):
    stock_data['Buy_Signal'] = np.where(
        (stock_data['SMA50'] > stock_data['SMA200']) &
        (stock_data['RSI'] < 30), 1, 0)
    
    stock_data['Sell_Signal'] = np.where(
        (stock_data['SMA50'] < stock_data['SMA200']) &
        (stock_data['RSI'] > 70), 1, 0)
    
    return stock_data

# Compile recommendations for multiple assets
def stock_recommendation(ticker, start_date, end_date):
    stock_data = fetch_stock_data(ticker, start_date, end_date)
    stock_data = calculate_technical_indicators(stock_data)
    stock_data = generate_signals(stock_data)
    
    latest_data = stock_data.iloc[-1]
    recommendation = "Hold"
    
    if latest_data['Buy_Signal'] == 1:
        recommendation = "Buy"
    elif latest_data['Sell_Signal'] == 1:
        recommendation = "Sell"
    
    return recommendation, stock_data

def pool_recommendation(tickers, start_date, end_date):
    recommendations = {}
    for ticker in tickers:
        rec, _ = stock_recommendation(ticker, start_date, end_date)
        recommendations[ticker] = rec
    return recommendations

# Example usage with more stocks
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NFLX", "FB", "NVDA", "BABA", "JPM", "V", "DIS", "PYPL", "INTC", "CSCO", "ADBE", "PFE", "KO", "PEP", "MRK"]
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

recommendations = pool_recommendation(tickers, start_date, end_date)
for ticker, rec in recommendations.items():
    print(f"Recommendation for {ticker}: {rec}")
